In [1]:
%load_ext tensorboard
import tensorflow
from datetime import datetime
from tensorflow import keras

#print(datetime.now)

In [2]:
import gym
import random
from envTSP import TSPDistCost

env = TSPDistCost()
states = env.observation_space.shape[0]
actions = env.action_space.n

logdir = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)



episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    path = []
    path.append(env.state[0])
    while not done:
        action = random.randint(0, env.N - 1)
        path.append(action)
        n_state, reward, done, info = env.step(action)
        score+=reward
    print(path)
    print('Episode:{} Score:{}'.format(episode, score))



[0, 11, 10, 8, 11]
Episode:1 Score:-292.36010517139727
[14, 14]
Episode:2 Score:-300
[10, 6, 11, 5, 6]
Episode:3 Score:-293.84491867928017
[0, 2, 3, 12, 0]
Episode:4 Score:-288.6474797916822
[10, 10]
Episode:5 Score:-300
[7, 6, 1, 6]
Episode:6 Score:-297.21293843617065
[1, 11, 3, 11]
Episode:7 Score:-294.4220982487686
[9, 4, 4]
Episode:8 Score:-298.72358599465406
[11, 1, 14, 5, 0, 9, 8, 2, 12, 6, 8]
Episode:9 Score:-269.3129478568221
[1, 9, 14, 6, 1]
Episode:10 Score:-290.54415897656054


2022-06-26 01:04:54.944233: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.


In [3]:
import numpy as np
import tensorflow.keras as keras
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Dropout, Input, Concatenate

from keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam


def build_model(states, actions):
    model = tensorflow.keras.Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model




Using TensorFlow backend.
/home/students/s290510/TFM/eki_tsp/ekit_venv/lib/python3.7/site-packages/keras/callbacks/callbacks.py:19: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [4]:
model = build_model(env.N +1, env.N)
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               2176      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 15)                1935      
Total params: 53,647
Trainable params: 53,647
Non-trainable params: 0
____________________________________________________

2022-06-26 01:05:01.800039: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-26 01:05:01.840746: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2300000000 Hz
2022-06-26 01:05:01.846094: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x563d8fe293a0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-06-26 01:05:01.846133: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [12]:


from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory



In [13]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=65000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10000, target_model_update=1e-2)
    return dqn

dqn = build_agent(model, actions)
loss_fn = keras.losses.SparseCategoricalCrossentropy()
dqn.compile(optimizer=Adam(learning_rate=0.002), metrics=['mae'])
dqn.fit(env, nb_steps=200000, visualize=True, verbose=1, callbacks=[tensorboard_callback]
)

Training for 200000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
10000/10000 [==============================] - 19s 2ms/step - reward: -99.1658
3333 episodes - episode_reward: -297.528 [-297.528, -295.421]

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 103s 10ms/step - reward: -56.8673
2135 episodes - episode_reward: -266.379 [-300.000, -243.135] - loss: 63.891 - mae: 237.689 - mean_q: -222.523

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 94s 9ms/step - reward: -31.1003
1262 episodes - episode_reward: -246.442 [-292.927, -231.514] - loss: 5.544 - mae: 251.412 - mean_q: -238.689

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 92s 9ms/step - reward: -22.8257
953 episodes - episode_reward: -239.472 [-291.822, -231.514] - loss: 5.208 - mae: 244.290 - mean_q: -230.855

Int

In [ ]:
#%load_ext tensorboard
%tensorboard --logdir logs/